<a href="https://colab.research.google.com/github/waltz2u/bd/blob/master/Movie_Rental_with_Spark_DataFrames.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Drive mounting

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

## Setting things up

In [0]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:13 https://developer.download.nvi

In [0]:
!ls

gdrive	sample_data  spark-2.4.5-bin-hadoop2.7	spark-2.4.5-bin-hadoop2.7.tgz


Switch to Java 8 if it is Java 11 now

In [0]:
!update-alternatives --config java

There are 2 choices for the alternative java (providing /usr/bin/java).

  Selection    Path                                            Priority   Status
------------------------------------------------------------
* 0            /usr/lib/jvm/java-11-openjdk-amd64/bin/java      1111      auto mode
  1            /usr/lib/jvm/java-11-openjdk-amd64/bin/java      1111      manual mode
  2            /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java   1081      manual mode

Press <enter> to keep the current choice[*], or type selection number: 2
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java to provide /usr/bin/java (java) in manual mode


Double check if it is Java 8 now

In [0]:
!java -version 

openjdk version "1.8.0_242"
OpenJDK Runtime Environment (build 1.8.0_242-8u242-b08-0ubuntu3~18.04-b08)
OpenJDK 64-Bit Server VM (build 25.242-b08, mixed mode)


Set some environment variables

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.2.3-bin-hadoop2.7"

## Spark DataFrame

In [0]:
import findspark
findspark.init("/content/spark-2.4.5-bin-hadoop2.7")
from pyspark.sql import SparkSession
# Sets the Spark master URL to connect to, such as "local" to run locally, 
# "local[4]" to run locally with 4 cores, or "spark://master:7077" to run on a Spark standalone cluster.
spark = SparkSession.builder.master("local[*]").getOrCreate() #

# another way using SparkConf
# from pyspark.conf import SparkConf
# from pyspark.context import SparkContext
# from pyspark.sql import SparkSession
# conf = SparkConf().setAppName("MLlib")
# sc = SparkContext(conf=conf)
# sc.setLogLevel("WARN")
# spark = SparkSession(sc)

Reading Data

In [0]:
payment_df = spark.read.format("csv").option("header", "true").load(r'/content/gdrive/My Drive/models/rental/payment.csv')
payment_df = payment_df.alias('payment_df')
payment_df.show(3)

+----------+-----------+--------+---------+------+-------------------+-------------------+
|payment_id|customer_id|staff_id|rental_id|amount|       payment_date|        last_update|
+----------+-----------+--------+---------+------+-------------------+-------------------+
|         1|          1|       1|       76|  2.99|2005-05-25 11:30:37|2006-02-15 22:12:30|
|         2|          1|       1|      573|  0.99|2005-05-28 10:35:23|2006-02-15 22:12:30|
|         3|          1|       1|     1185|  5.99|2005-06-15 00:54:12|2006-02-15 22:12:30|
+----------+-----------+--------+---------+------+-------------------+-------------------+
only showing top 3 rows



In [0]:
rental_df = spark.read.format("csv").option("header", "true").load(r'/content/gdrive/My Drive/models/rental/rental.csv')
rental_df = rental_df.alias('rental_df')
rental_df.show(3)

+---------+-------------------+------------+-----------+-------------------+--------+-------------------+
|rental_id|        rental_date|inventory_id|customer_id|        return_date|staff_id|        last_update|
+---------+-------------------+------------+-----------+-------------------+--------+-------------------+
|        1|2005-05-24 22:53:30|         367|        130|2005-05-26 22:04:30|       1|2006-02-15 21:30:53|
|        2|2005-05-24 22:54:33|        1525|        459|2005-05-28 19:40:33|       1|2006-02-15 21:30:53|
|        3|2005-05-24 23:03:39|        1711|        408|2005-06-01 22:12:39|       1|2006-02-15 21:30:53|
+---------+-------------------+------------+-----------+-------------------+--------+-------------------+
only showing top 3 rows



In [0]:
staff_df = spark.read.format("csv").option("header", "true").load(r'/content/gdrive/My Drive/models/rental/staff.csv')
staff_df = staff_df.alias('staff_df')
staff_df.show(3)

+--------+----------+---------+----------+-------+--------------------+--------+------+--------+--------------------+--------------+
|staff_id|first_name|last_name|address_id|picture|               email|store_id|active|username|            password|   last_update|
+--------+----------+---------+----------+-------+--------------------+--------+------+--------+--------------------+--------------+
|       1|      Mike|  Hillyer|         3|   NULL|Mike.Hillyer@saki...|       1|     1|    Mike|8cb2237d0679ca88d...|2/15/2006 3:57|
|       2|       Jon| Stephens|         4|   NULL|Jon.Stephens@saki...|       2|     1|     Jon|                NULL|2/15/2006 3:57|
+--------+----------+---------+----------+-------+--------------------+--------+------+--------+--------------------+--------------+



In [0]:
inventory_df = spark.read.format("csv").option("header", "true").load(r'/content/gdrive/My Drive/models/rental/inventory.csv')
inventory_df = inventory_df.alias('inventory_df')
inventory_df.show(3)

+------------+-------+--------+-------------------+
|inventory_id|film_id|store_id|        last_update|
+------------+-------+--------+-------------------+
|           1|      1|       1|2006-02-15 05:09:17|
|           2|      1|       1|2006-02-15 05:09:17|
|           3|      1|       1|2006-02-15 05:09:17|
+------------+-------+--------+-------------------+
only showing top 3 rows



In [0]:
store_df = spark.read.format("csv").option("header", "true").load(r'/content/gdrive/My Drive/models/rental/store.csv')
store_df = store_df.alias('store_df')
store_df.show(3)

+--------+----------------+----------+-------------------+
|store_id|manager_staff_id|address_id|        last_update|
+--------+----------------+----------+-------------------+
|       1|               1|         1|2006-02-15 04:57:12|
|       2|               2|         2|2006-02-15 04:57:12|
+--------+----------------+----------+-------------------+



In [0]:
address_df = spark.read.format("csv").option("header", "true").load(r'/content/gdrive/My Drive/models/rental/address.csv')
address_df = address_df.alias('address_df')
address_df.show(3)

+----------+------------------+--------+--------+-------+-----------+-----------+-------------------+
|address_id|           address|address2|district|city_id|postal_code|      phone|        last_update|
+----------+------------------+--------+--------+-------+-----------+-----------+-------------------+
|         1| 47 MySakila Drive|    NULL| Alberta|    300|       null|       null|2014-09-25 22:30:27|
|         2|28 MySQL Boulevard|    NULL|     QLD|    576|       null|       null|2014-09-25 22:30:09|
|         3| 23 Workhaven Lane|    NULL| Alberta|    300|       null|14033335568|2014-09-25 22:30:27|
+----------+------------------+--------+--------+-------+-----------+-----------+-------------------+
only showing top 3 rows



In [0]:
customer_df = spark.read.format("csv").option("header", "true").load(r'/content/gdrive/My Drive/models/rental/customer.csv')
customer_df = customer_df.alias('customer_df')
customer_df.show(3)

+-----------+--------+----------+---------+--------------------+----------+------+-------------------+-------------------+
|customer_id|store_id|first_name|last_name|               email|address_id|active|        create_date|        last_update|
+-----------+--------+----------+---------+--------------------+----------+------+-------------------+-------------------+
|          1|       1|      MARY|    SMITH|MARY.SMITH@sakila...|         5|     1|2006-02-14 22:04:36|2006-02-15 04:57:20|
|          2|       1|  PATRICIA|  JOHNSON|PATRICIA.JOHNSON@...|         6|     1|2006-02-14 22:04:36|2006-02-15 04:57:20|
|          3|       1|     LINDA| WILLIAMS|LINDA.WILLIAMS@sa...|         7|     1|2006-02-14 22:04:36|2006-02-15 04:57:20|
+-----------+--------+----------+---------+--------------------+----------+------+-------------------+-------------------+
only showing top 3 rows



In [0]:
film_df = spark.read.format("csv").option("header", "true").load(r'/content/gdrive/My Drive/models/rental/film.csv')
film_df = film_df.alias('film_df')
film_df.show(3)

+-------+----------------+--------------------+------------+-----------+--------------------+---------------+-----------+------+----------------+------+--------------------+-------------------+
|film_id|           title|         description|release_year|language_id|original_language_id|rental_duration|rental_rate|length|replacement_cost|rating|    special_features|        last_update|
+-------+----------------+--------------------+------------+-----------+--------------------+---------------+-----------+------+----------------+------+--------------------+-------------------+
|      1|ACADEMY DINOSAUR|A Epic Drama of a...|        2006|          1|                NULL|              6|       0.99|    86|           20.99|    PG|Deleted Scenes,Be...|2006-02-15 05:03:42|
|      2|  ACE GOLDFINGER|A Astounding Epis...|        2006|          1|                NULL|              3|       4.99|    48|           12.99|     G|Trailers,Deleted ...|2006-02-15 05:03:42|
|      3|ADAPTATION HOLES|A As

Create a data frame based on the payment data frame. It contains a list of customer_id and total payment amount grouped by
customer_id, sorted by the total amount in descending order. Other columns may be included as needed. 

In [0]:
cust_amount_df = payment_df.withColumn("amount", payment_df["amount"].cast("double")).groupby('customer_id').sum().orderBy('sum(amount)',ascending = False)
cust_amount_df.show(3)

+-----------+------------------+
|customer_id|       sum(amount)|
+-----------+------------------+
|        526| 221.5500000000001|
|        148| 216.5400000000001|
|        144|195.58000000000007|
+-----------+------------------+
only showing top 3 rows



Create a data frame based on the rental data frame. It contains a list of rentals from the rental data set that were rented in 2005 ordered on return_date in ascending order. Other columns may be included as needed. 

In [0]:
rental_2005_df = rental_df.where("rental_date >= '2005-01-01 00:00:00' and rental_date <= '2005-12-31 11:59:59'").orderBy('return_date',ascending = True)
rental_2005_df.show(3)

+---------+-------------------+------------+-----------+-------------------+--------+-------------------+
|rental_id|        rental_date|inventory_id|customer_id|        return_date|staff_id|        last_update|
+---------+-------------------+------------+-----------+-------------------+--------+-------------------+
|       32|2005-05-25 04:06:21|        3832|        230|2005-05-25 23:55:21|       1|2006-02-15 21:30:53|
|       21|2005-05-25 01:59:46|         146|        388|2005-05-26 01:01:46|       2|2006-02-15 21:30:53|
|       14|2005-05-25 00:31:15|        2701|        446|2005-05-26 02:56:15|       1|2006-02-15 21:30:53|
+---------+-------------------+------------+-----------+-------------------+--------+-------------------+
only showing top 3 rows



Create a data frame based on rental data frame. It contains a list of rental_id count from rental data set grouped by staff_id. 

In [0]:
cust_amount_df = rental_df.groupby('staff_id').count()
cust_amount_df.show(3)

+--------+-----+
|staff_id|count|
+--------+-----+
|       1| 8040|
|       2| 8004|
+--------+-----+



Now add staff first and last name.

In [0]:
cust_amount_df = rental_df.groupby('staff_id').count().join(staff_df, rental_df.staff_id ==
staff_df.staff_id,"inner").select('first_name', 'last_name','count')
cust_amount_df.show(3)

+----------+---------+-----+
|first_name|last_name|count|
+----------+---------+-----+
|      Mike|  Hillyer| 8040|
|       Jon| Stephens| 8004|
+----------+---------+-----+



 Create a data frame based on rental data frame, joined with inventory, store, and address data frames to get store address, joined with customer data frame to get customer first and last name, joined with staff data frame to get customer first and last name, joined with film data set to get film name and description, and ordered by return_date in the ascending order. 

All columns 

In [0]:
combined_df = rental_df.join(inventory_df, rental_df.inventory_id == inventory_df.inventory_id).join(store_df, inventory_df.store_id == store_df.store_id) \
.join(address_df, store_df.address_id == address_df.address_id).join(customer_df, rental_df.customer_id == customer_df.customer_id) \
.join(staff_df, rental_df.staff_id == staff_df.staff_id).join(film_df, inventory_df.film_id == film_df.film_id).orderBy('return_date') 
combined_df.show(3)

+---------+-------------------+------------+-----------+-------------------+--------+-------------------+------------+-------+--------+-------------------+--------+----------------+----------+-------------------+----------+------------------+--------+--------+-------+-----------+-----+-------------------+-----------+--------+----------+---------+--------------------+----------+------+-------------------+-------------------+--------+----------+---------+----------+-------+--------------------+--------+------+--------+--------------------+--------------+-------+------------------+--------------------+------------+-----------+--------------------+---------------+-----------+------+----------------+------+--------------------+-------------------+
|rental_id|        rental_date|inventory_id|customer_id|        return_date|staff_id|        last_update|inventory_id|film_id|store_id|        last_update|store_id|manager_staff_id|address_id|        last_update|address_id|           address|addre

Selected columns

In [0]:
combined_df = rental_df.join(inventory_df, rental_df.inventory_id == inventory_df.inventory_id).join(store_df, inventory_df.store_id == store_df.store_id) \
.join(address_df, store_df.address_id == address_df.address_id).join(customer_df, rental_df.customer_id == customer_df.customer_id) \
.join(staff_df, rental_df.staff_id == staff_df.staff_id).join(film_df, inventory_df.film_id == film_df.film_id).orderBy('return_date') \
.select('rental_date','return_date','address','title', 'description','customer_df.first_name','customer_df.last_name', 'staff_df.first_name','staff_df.last_name') 
combined_df.show(3)

+-------------------+-------------------+------------------+------------------+--------------------+----------+---------+----------+---------+
|        rental_date|        return_date|           address|             title|         description|first_name|last_name|first_name|last_name|
+-------------------+-------------------+------------------+------------------+--------------------+----------+---------+----------+---------+
|2005-05-25 04:06:21|2005-05-25 23:55:21|28 MySQL Boulevard| STALLION SUNDANCE|A Fast-Paced Tale...|       JOY|   GEORGE|      Mike|  Hillyer|
|2005-05-25 01:59:46|2005-05-26 01:01:46| 47 MySakila Drive|     APACHE DIVINE|A Awe-Inspiring R...|     CRAIG|  MORRELL|       Jon| Stephens|
|2005-05-25 00:31:15|2005-05-26 02:56:15|28 MySQL Boulevard|MONTEREY LABYRINTH|A Awe-Inspiring D...|  THEODORE|     CULP|      Mike|  Hillyer|
+-------------------+-------------------+------------------+------------------+--------------------+----------+---------+----------+---------+

Create a data frame that contains a list of customer_id whose sum amount of payment in 2005 is more than $200. 

In [0]:
cust_sum_2005_200_df = payment_df.withColumn("amount", payment_df["amount"].cast("double")).where("payment_date >= '2005-01-01 00:00:00' and payment_date <= '2005-12-31 11:59:59'") \
.groupby('customer_id').sum('amount').where('sum(amount) > 200')
cust_sum_2005_200_df.show()

+-----------+-----------------+
|customer_id|      sum(amount)|
+-----------+-----------------+
|        526|221.5500000000001|
|        148|216.5400000000001|
+-----------+-----------------+



Join the result with customer dataframe  to get their names

In [0]:
cust_sum_2005_200_df = cust_sum_2005_200_df.join(customer_df, cust_sum_2005_200_df.customer_id == customer_df.customer_id).select('first_name','last_name','sum(amount)')
cust_sum_2005_200_df.show()

+----------+---------+-----------------+
|first_name|last_name|      sum(amount)|
+----------+---------+-----------------+
|      KARL|     SEAL|221.5500000000001|
|   ELEANOR|     HUNT|216.5400000000001|
+----------+---------+-----------------+



Create a data frame that contains a list of customer_id that rented in 2005; Create another data frame that contains a list of customer_id that rented in 2006 (filter using rental_date). 

In [0]:
cust_2005_df = rental_df.where("rental_date >= '2005-01-01 00:00:00' and rental_date <= '2005-12-31 11:59:59'")
cust_2005_df.show(3)

+---------+-------------------+------------+-----------+-------------------+--------+-------------------+
|rental_id|        rental_date|inventory_id|customer_id|        return_date|staff_id|        last_update|
+---------+-------------------+------------+-----------+-------------------+--------+-------------------+
|        1|2005-05-24 22:53:30|         367|        130|2005-05-26 22:04:30|       1|2006-02-15 21:30:53|
|        2|2005-05-24 22:54:33|        1525|        459|2005-05-28 19:40:33|       1|2006-02-15 21:30:53|
|        3|2005-05-24 23:03:39|        1711|        408|2005-06-01 22:12:39|       1|2006-02-15 21:30:53|
+---------+-------------------+------------+-----------+-------------------+--------+-------------------+
only showing top 3 rows



In [0]:
cust_2006_df = rental_df.where("rental_date >= '2006-01-01 00:00:00' and rental_date <= '2006-12-31 11:59:59'")
cust_2006_df.show(3)

+---------+-------------------+------------+-----------+-----------+--------+-------------------+
|rental_id|        rental_date|inventory_id|customer_id|return_date|staff_id|        last_update|
+---------+-------------------+------------+-----------+-----------+--------+-------------------+
|    11496|2006-02-14 15:16:03|        2047|        155|       NULL|       1|2006-02-15 21:30:53|
|    11541|2006-02-14 15:16:03|        2026|        335|       NULL|       1|2006-02-15 21:30:53|
|    11563|2006-02-14 15:16:03|        1545|         83|       NULL|       1|2006-02-15 21:30:53|
+---------+-------------------+------------+-----------+-----------+--------+-------------------+
only showing top 3 rows



Now create a data frame of customer_id that exist in the 2005 data frame but not in the 2006 data frame. 

In [0]:
cust_2005_not_2006_df = cust_2005_df.select('customer_id').subtract(cust_2006_df.select('customer_id'))
cust_2005_not_2006_df.distinct().count()

441

Now create a data frame of customer_id that exist in the
2006 data frame but not in the 2005 data frame

In [0]:
cust_2006_not_2005_df = cust_2006_df.select('customer_id').subtract(cust_2005_df.select('customer_id'))
cust_2006_not_2005_df.distinct().show()

+-----------+
|customer_id|
+-----------+
+-----------+

